In [ ]:
!gdown  --id '1-0Ld2ewmzMgN_6A6AP175QTLSsjspS_Y/view?usp=sharing'

Permission denied: https://drive.google.com/uc?id=1-0Ld2ewmzMgN_6A6AP175QTLSsjspS_Y/view?usp=sharing
Maybe you need to change permission over 'Anyone with the link'?


In [1]:
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm

In [32]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
#from keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
data=pd.DataFrame(columns=['X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'X3', 'Y3', 'Z3', 'X4', 'Y4', 'Z4',
       'X5', 'Y5', 'Z5', 'X6', 'Y6', 'Z6', 'X7', 'Y7', 'Z7', 'X8', 'Y8', 'Z8',
       'X9', 'Y9', 'Z9', 'X10', 'Y10', 'Z10', 'X11', 'Y11', 'Z11', 'X12',
       'Y12', 'Z12', 'X13', 'Y13', 'Z13','subject_id','activity'])

In [4]:


def segmentation(df, overlap_rate, time_window):
    
    # make a list for segment window and its label
    seg_data = []

    #convert overlap rate to step for sliding window
    overlap = int((1 - overlap_rate)*time_window)
    
    # interpolate
    df = df.interpolate().ffill().fillna(0)
    #segment
    for i in range(0, len(df)-time_window+1, overlap):
        seg_data.append(df.loc[i:i+time_window-1, :].copy().reset_index(drop=True))
        
    return seg_data



In [7]:
data_list = []
for subjects in tqdm(glob.glob("/content/drive/MyDrive/Competitions/2021/bento/data/TrainData/*")):
    for activity in tqdm(glob.glob(subjects+'/*')):
       for file in tqdm(glob.glob(activity+'/*')):
         temp = pd.read_csv(file)
         temp.drop('subject_id',axis=1,inplace=True)
         data_list.extend(segmentation(temp, 0.5, 500))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

 33%|███▎      | 2/6 [00:00<00:00, 14.31it/s]

100%|██████████| 6/6 [00:00<00:00, 15.88it/s]

 10%|█         | 1/10 [00:00<00:03,  2.59it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

 40%|████      | 2/5 [00:00<00:00, 16.02it/s]

100%|██████████| 5/5 [00:00<00:00, 15.27it/s]

 20%|██        | 2/10 [00:00<00:02,  2.67it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

 40%|████      | 2/5 [00:00<00:00, 15.45it/s]

100%|██████████| 5/5 [00:00<00:00, 15.16it/s]

 30%|███       | 3/10 [00:01<00:02,  2.69it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

 40%|████      | 2/5 [00:00<00:00, 14.14it/s]

100%|██████████| 5/5 [00:00<00:00, 14.03it/s]

 40%|████      | 4/10 [00:01<00:02,  2.69it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

 40%|████      | 2/5 [00:00<00:00, 14.11it/s]

100%|██████████| 5/5 [00:00<00:00, 14.08it/s]

 50%|█████     | 5/10 [00:01<00:01,  2.70it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

 40%|████      |

In [45]:
seg=[]
labels=[]
for i in tqdm(range(len(data_list))):
  seg.append(data_list[i].iloc[:,:-1])
  labels.append(data_list[i].iloc[0,-1])

100%|██████████| 3573/3573 [00:02<00:00, 1638.28it/s]


In [23]:
seg[0].shape

(500, 39)

In [49]:
seg=np.array(seg)

In [46]:
labels=np.array(labels)

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split




In [61]:
x_train, x_val, y_train, y_val = train_test_split(seg, labels, test_size=.20, random_state=12)

In [62]:
x_train = x_train/x_train.max(axis = 0)
x_val = x_val/x_val.max(axis = 0)

In [63]:
y_train.shape

(2858,)

In [65]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
y_train = onehot_encoder.fit_transform(np.array(y_train).reshape(-1,1))
y_val = onehot_encoder.fit_transform(np.array(y_val).reshape(-1,1))

In [66]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(500,39)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [67]:


model.fit(x_train,y_train,epochs = 50,verbose = 2,validation_data = (x_val,y_val))



Epoch 1/50
90/90 - 14s - loss: 1.9188 - accuracy: 0.2575 - val_loss: 1.5307 - val_accuracy: 0.3944
Epoch 2/50
90/90 - 8s - loss: 1.5113 - accuracy: 0.3936 - val_loss: 1.4495 - val_accuracy: 0.3860
Epoch 3/50
90/90 - 8s - loss: 1.3682 - accuracy: 0.4461 - val_loss: 1.3821 - val_accuracy: 0.4573
Epoch 4/50
90/90 - 8s - loss: 1.3148 - accuracy: 0.4713 - val_loss: 1.3350 - val_accuracy: 0.4643
Epoch 5/50
90/90 - 8s - loss: 1.2644 - accuracy: 0.4923 - val_loss: 1.4537 - val_accuracy: 0.4238
Epoch 6/50
90/90 - 8s - loss: 1.1666 - accuracy: 0.5308 - val_loss: 1.3174 - val_accuracy: 0.4965
Epoch 7/50
90/90 - 8s - loss: 1.1387 - accuracy: 0.5420 - val_loss: 1.4262 - val_accuracy: 0.4503
Epoch 8/50
90/90 - 8s - loss: 1.0857 - accuracy: 0.5714 - val_loss: 1.3153 - val_accuracy: 0.4881
Epoch 9/50
90/90 - 8s - loss: 1.0480 - accuracy: 0.5752 - val_loss: 1.2486 - val_accuracy: 0.5217
Epoch 10/50
90/90 - 8s - loss: 0.9841 - accuracy: 0.6018 - val_loss: 1.2104 - val_accuracy: 0.5217
Epoch 11/50
90/90 